## Use case #1: using file inputParams.json (recommended use)

Execute the following cells only the first time running this notebook:

In [1]:
import os
os.chdir("..")

import pandas as pd
pd.set_option('display.precision', 2)

Load the input parameters file in a new instance of the `controller` an request the first `DataFrame` to the specified data source.

In [2]:
from synthDataGen.base import Controller

controller = Controller()
controller.loadMainParams("inputParams.json", "./synthDataGen/settings/")
df = controller.getDataFromSource()
df

,2015,2016,2017,2018,2019,2020,2021,2022
dateNoYear,,,,,,,,
2023-01-01 00:00:00,NaN,3.6,3.6,5.4,1.8,3.6,9.4,9.4
2023-01-01 01:00:00,NaN,0.0,1.8,3.6,0.0,2.9,14.8,8.6
2023-01-01 02:00:00,NaN,3.6,1.8,9.4,1.8,3.6,7.6,7.9
2023-01-01 03:00:00,NaN,1.8,3.6,7.6,1.8,3.6,11.2,7.2
2023-01-01 04:00:00,NaN,1.8,3.6,5.4,3.6,3.6,7.6,6.8
...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,4.7,3.6,11.2,1.3,3.6,12.1,10.4,0.7
2023-12-31 20:00:00,4.3,3.6,9.4,1.8,5.4,9.4,10.8,3.6
2023-12-31 21:00:00,3.6,1.8,9.4,1.8,3.6,9.4,10.8,1.8


Adjust the source `DataFrame` with the anual adjustments dictionary and resample it to the desired resolution:

In [3]:
from synthDataGen.base import Adjustments

adjustments = Adjustments(controller.inputJSON)
df = adjustments.performAnualAdjustments(df)
df

Adjusting years: 2015,2016,2017,2018,2019,2020,2021,2022


,2015,2016,2017,2018,2019,2020,2021,2022
dateNoYear,,,,,,,,
2023-01-01 00:00:00,NaN,3.96,3.96,5.94,1.98,3.96,10.34,10.34
2023-01-01 01:00:00,NaN,0.00,1.98,3.96,0.00,3.19,16.28,9.46
2023-01-01 02:00:00,NaN,3.96,1.98,10.34,1.98,3.96,8.36,8.69
2023-01-01 03:00:00,NaN,1.98,3.96,8.36,1.98,3.96,12.32,7.92
2023-01-01 04:00:00,NaN,1.98,3.96,5.94,3.96,3.96,8.36,7.48
...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,5.17,3.96,12.32,1.43,3.96,13.31,11.44,0.77
2023-12-31 20:00:00,4.73,3.96,10.34,1.98,5.94,10.34,11.88,3.96
2023-12-31 21:00:00,3.96,1.98,10.34,1.98,3.96,10.34,11.88,1.98


In [4]:
df = adjustments.upsample(df)
df

,2015,2016,2017,2018,2019,2020,2021,2022
2023-01-01 00:00:00,NaN,3.96e+00,3.96,5.94,1.98,3.96,10.34,10.34
2023-01-01 00:15:00,NaN,1.98e+00,3.29,4.38,1.00,3.58,13.60,10.11
2023-01-01 00:30:00,NaN,6.62e-01,2.74,3.53,0.35,3.33,15.68,9.88
2023-01-01 00:45:00,NaN,1.76e-03,2.30,3.39,0.01,3.20,16.57,9.67
2023-01-01 01:00:00,NaN,0.00e+00,1.98,3.96,0.00,3.19,16.28,9.46
...,...,...,...,...,...,...,...,...
2023-12-31 22:00:00,3.96,1.98e+00,8.36,1.98,1.98,14.30,11.88,3.96
2023-12-31 22:15:00,4.25,2.58e+00,7.76,1.98,1.78,14.52,11.77,4.27
2023-12-31 22:30:00,4.68,3.44e+00,7.16,1.98,1.71,14.26,11.60,4.38
2023-12-31 22:45:00,5.24,4.56e+00,6.55,1.98,1.78,13.53,11.38,4.27


In [5]:
df = adjustments.downsample(df)
df

,2015,2016,2017,2018,2019,2020,2021,2022
2023-01-01 00:00:00,NaN,1.57,2.44,5.36,0.77,3.49,13.62,9.58
2023-01-01 02:00:00,NaN,2.64,3.44,8.52,2.39,3.99,10.29,8.06
2023-01-01 04:00:00,NaN,2.00,2.54,9.81,2.62,4.38,11.44,7.82
2023-01-01 06:00:00,NaN,1.59,1.99,8.35,1.26,2.75,12.87,3.13
2023-01-01 08:00:00,NaN,1.45,2.29,5.20,0.41,2.83,13.49,1.21
...,...,...,...,...,...,...,...,...
2023-12-31 14:00:00,11.39,10.11,16.76,3.89,5.48,21.21,3.96,4.88
2023-12-31 16:00:00,3.80,7.23,16.23,1.18,6.30,21.05,2.70,4.07
2023-12-31 18:00:00,5.40,4.50,13.12,1.51,4.24,12.25,9.43,1.83
2023-12-31 20:00:00,4.14,2.49,10.04,1.99,4.26,10.91,11.87,2.93


Get new samples from the current `DataFrame`:

In [6]:
from synthDataGen.base import Sampling

sampling = Sampling(controller.inputJSON)
samples = sampling.getSamples(df)
samples.head()

c:\Users\German\Documents\Work\Repsol\synthDataGen\synthDataGen\base.py:579: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  resultingDataFrame[index] = samples


,2023-01-01 00:00:00,2023-01-01 02:00:00,2023-01-01 04:00:00,2023-01-01 06:00:00,2023-01-01 08:00:00,2023-01-01 10:00:00,2023-01-01 12:00:00,2023-01-01 14:00:00,2023-01-01 16:00:00,2023-01-01 18:00:00,...,2023-12-31 04:00:00,2023-12-31 06:00:00,2023-12-31 08:00:00,2023-12-31 10:00:00,2023-12-31 12:00:00,2023-12-31 14:00:00,2023-12-31 16:00:00,2023-12-31 18:00:00,2023-12-31 20:00:00,2023-12-31 22:00:00
0,8.81,11.01,8.65,6.60,3.17,1.99,1.29,12.46,3.60,8.17,...,1.61,1.12,6.59,8.35,8.16,11.46,13.55,9.30,10.29,9.34
1,3.66,9.90,8.35,4.38,4.60,15.75,15.87,0.60,26.41,12.47,...,6.48,6.24,8.27,4.34,10.50,11.05,10.06,12.16,7.28,11.71
2,3.65,4.10,7.24,7.52,3.52,3.33,3.09,4.67,16.35,3.67,...,4.96,6.35,4.83,3.48,11.22,15.30,5.00,4.15,9.91,1.78
3,12.06,4.16,12.69,7.32,2.04,0.78,5.77,14.73,7.20,2.55,...,4.91,7.11,3.75,3.19,0.33,3.81,9.62,7.49,9.79,10.21
4,4.66,9.15,3.31,10.15,6.45,9.54,11.99,0.57,15.91,1.92,...,4.39,6.12,0.48,2.53,9.83,12.07,4.91,4.40,0.03,7.18


# Use case #2: passing parameters


In [7]:
from datetime import datetime

from synthDataGen.base import Controller

controller = Controller()
controller.loadMainParams("inputParams.json", "./synthDataGen/settings/")
df = controller.getDataFromSource(initialYear=2018, initDatetime=datetime(2023, 6, 5, 7, 0), hoursAhead=6)
df

,2015,2016,2017,2018,2019,2020,2021,2022
dateNoYear,,,,,,,,
2023-01-01 00:00:00,NaN,3.6,3.6,5.4,1.8,3.6,9.4,9.4
2023-01-01 01:00:00,NaN,0.0,1.8,3.6,0.0,2.9,14.8,8.6
2023-01-01 02:00:00,NaN,3.6,1.8,9.4,1.8,3.6,7.6,7.9
2023-01-01 03:00:00,NaN,1.8,3.6,7.6,1.8,3.6,11.2,7.2
2023-01-01 04:00:00,NaN,1.8,3.6,5.4,3.6,3.6,7.6,6.8
...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,4.7,3.6,11.2,1.3,3.6,12.1,10.4,0.7
2023-12-31 20:00:00,4.3,3.6,9.4,1.8,5.4,9.4,10.8,3.6
2023-12-31 21:00:00,3.6,1.8,9.4,1.8,3.6,9.4,10.8,1.8


In [8]:
from synthDataGen.base import Adjustments

adjustments = Adjustments(controller.inputJSON)
df = adjustments.performAnualAdjustments(df, adjustmentsDict={2018: 1.2, 2019: 2.3, 2020: 1.45, 2021: 3, 2022: 8})
df

Adjusting years: 2018,2019,2020,2021,2022


,2015,2016,2017,2018,2019,2020,2021,2022
dateNoYear,,,,,,,,
2023-01-01 00:00:00,NaN,3.6,3.6,5.46,1.84,3.65,9.68,10.15
2023-01-01 01:00:00,NaN,0.0,1.8,3.64,0.00,2.94,15.24,9.29
2023-01-01 02:00:00,NaN,3.6,1.8,9.51,1.84,3.65,7.83,8.53
2023-01-01 03:00:00,NaN,1.8,3.6,7.69,1.84,3.65,11.54,7.78
2023-01-01 04:00:00,NaN,1.8,3.6,5.46,3.68,3.65,7.83,7.34
...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,4.7,3.6,11.2,1.32,3.68,12.28,10.71,0.76
2023-12-31 20:00:00,4.3,3.6,9.4,1.82,5.52,9.54,11.12,3.89
2023-12-31 21:00:00,3.6,1.8,9.4,1.82,3.68,9.54,11.12,1.94


In [9]:
df = adjustments.upsample(df, frequency="20T", method="spline", order=3)
df

,2015,2016,2017,2018,2019,2020,2021,2022
2023-01-01 00:00:00,NaN,3.60,3.60,5.46,1.84,3.65,9.68,10.15
2023-01-01 00:20:00,NaN,0.81,2.95,2.80,1.13,3.31,14.70,9.84
2023-01-01 00:40:00,NaN,-0.15,2.85,2.51,1.35,3.01,16.05,9.54
2023-01-01 01:00:00,NaN,0.00,1.80,3.64,0.00,2.94,15.24,9.29
2023-01-01 01:20:00,NaN,1.19,2.69,5.85,1.71,2.84,12.55,9.02
...,...,...,...,...,...,...,...,...
2023-12-31 21:40:00,3.32,1.69,8.18,1.83,3.03,11.40,11.03,3.36
2023-12-31 22:00:00,3.60,1.80,7.60,1.82,1.84,13.19,11.12,3.89
2023-12-31 22:20:00,3.84,2.76,7.09,1.82,2.36,11.78,10.58,3.74
2023-12-31 22:40:00,4.49,3.82,6.33,1.82,1.96,11.86,10.47,3.86


In [10]:
df = adjustments.downsample(df, frequency="22.73T", aggregationFunc="mean")
df

,2015,2016,2017,2018,2019,2020,2021,2022
2023-01-01 00:00:00.000,NaN,2.20,3.28,4.13,1.48,3.48,12.19,9.99
2023-01-01 00:22:43.800,NaN,-0.15,2.85,2.51,1.35,3.01,16.05,9.54
2023-01-01 00:45:27.600,NaN,0.00,1.80,3.64,0.00,2.94,15.24,9.29
2023-01-01 01:08:11.400,NaN,1.19,2.69,5.85,1.71,2.84,12.55,9.02
2023-01-01 01:30:55.200,NaN,2.39,2.62,7.97,1.84,2.93,10.01,8.78
...,...,...,...,...,...,...,...,...
2023-12-31 21:06:40.800,3.37,1.59,8.59,1.83,3.31,11.15,11.28,3.13
2023-12-31 21:29:24.600,3.32,1.69,8.18,1.83,3.03,11.40,11.03,3.36
2023-12-31 21:52:08.400,3.60,1.80,7.60,1.82,1.84,13.19,11.12,3.89
2023-12-31 22:14:52.200,3.84,2.76,7.09,1.82,2.36,11.78,10.58,3.74


In [11]:
from synthDataGen.base import Sampling

sampling = Sampling(controller.inputJSON)
df = sampling.getSamples(df, 1000, "truncnorm")
df.head()

c:\Users\German\Documents\Work\Repsol\synthDataGen\synthDataGen\base.py:579: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  resultingDataFrame[index] = samples


,2023-01-01 00:00:00.000,2023-01-01 00:22:43.800,2023-01-01 00:45:27.600,2023-01-01 01:08:11.400,2023-01-01 01:30:55.200,2023-01-01 01:53:39.000,2023-01-01 02:16:22.800,2023-01-01 02:39:06.600,2023-01-01 03:01:50.400,2023-01-01 03:24:34.200,...,2023-12-31 19:13:01.800,2023-12-31 19:35:45.600,2023-12-31 19:58:29.400,2023-12-31 20:21:13.200,2023-12-31 20:43:57.000,2023-12-31 21:06:40.800,2023-12-31 21:29:24.600,2023-12-31 21:52:08.400,2023-12-31 22:14:52.200,2023-12-31 22:37:36.000
0,4.13,9.16,12.52,4.30,5.14,6.35,5.61,10.61,7.60,9.76,...,4.76,9.61,4.26,3.76,8.80,4.43,6.86,10.89,10.18,9.15
1,5.88,10.22,10.34,6.09,11.19,5.58,6.06,7.45,1.98,4.14,...,0.04,3.40,4.20,2.99,5.92,5.57,4.59,5.76,5.59,7.54
2,7.03,2.04,3.19,1.70,3.42,5.71,8.70,2.91,3.82,3.37,...,6.74,5.73,4.42,8.96,4.48,2.94,10.74,6.03,1.68,6.41
3,6.43,2.58,5.07,5.14,11.13,1.21,3.35,5.11,7.04,4.63,...,10.39,3.86,9.20,4.24,2.26,3.08,7.69,5.16,8.09,6.55
4,4.73,12.26,8.88,4.13,3.89,1.56,5.93,0.98,8.04,5.86,...,3.68,5.13,9.69,8.04,5.44,3.14,5.11,1.75,4.78,0.62
